In [5]:
import torch
from datasets import load_dataset, concatenate_datasets, DatasetDict
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification, BertForSequenceClassification, DataCollatorWithPadding

import math
import numpy as np
import matplotlib.pyplot as plt
import evaluate
import pandas as pd
import scipy
dataset = load_dataset("banking77").rename_column("label", "labels")

atis_full = load_dataset("tuetschek/atis").rename_column("intent", "labels").remove_columns(["slots", "id"])
atis_full = concatenate_datasets([atis_full['train'], atis_full['test']])
atis_full = atis_full.class_encode_column('labels')


# hyperparameters from https://github.com/clinc/oos-eval/blob/master/hyperparameters.csv
learning_rate = 4.00e-05
warmup_proportion = 0.1
train_batch_size = 16
eval_batch_size = 16
num_train_epochs = 5
gradient_accumulation_steps = 1

huggingface_modelname = "bert-large-uncased"

/Users/floris/.cache/huggingface/modules/datasets_modules/datasets/banking77/9898c11f6afa9521953d2ef205667b527bad14ef9cab445d470f16240c8c8ec4/banking77.py:59: FutureWarning: Dataset 'banking77' is deprecated and will be deleted. Use 'PolyAI/banking77' instead.
  warnings.warn(


In [7]:
dataset_full = concatenate_datasets([dataset["train"], dataset["test"]]).shuffle(seed=42)
train_size, cal_size, test_size = .6, .2, .2
train_test_set = dataset_full.train_test_split(train_size=train_size, stratify_by_column='labels', seed=42)
test_val_set = train_test_set["test"].train_test_split(train_size=test_size/(cal_size+test_size), stratify_by_column='labels', seed=42)
dataset = DatasetDict({
    'train': train_test_set['train'],
    'test': test_val_set['train'],
    'calibration': test_val_set['test']
})

assert math.fabs(((dataset['train'].num_rows / dataset_full.num_rows) - train_size)) < 1e-4
assert math.fabs(((dataset['test'].num_rows / dataset_full.num_rows) - test_size)) < 1e-4
assert math.fabs(((dataset['calibration'].num_rows / dataset_full.num_rows) - cal_size)) < 1e-4


In [8]:
math.fabs(((dataset['train'].num_rows / dataset_full.num_rows) - train_size))

6.114805472745566e-05

In [9]:
dataset_full

Dataset({
    features: ['text', 'labels'],
    num_rows: 13083
})

In [10]:
id2label = {i: name for i, name in enumerate(dataset["train"].features["labels"].names)}
label2id = {name: i for i, name in id2label.items()}

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 7849
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 2617
    })
    calibration: Dataset({
        features: ['text', 'labels'],
        num_rows: 2617
    })
})

In [12]:
torch.mps.current_allocated_memory()

0

In [13]:
tokenizer = AutoTokenizer.from_pretrained(huggingface_modelname)

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_atis = atis_full.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

assert dataset["train"].features["labels"].names == dataset["test"].features["labels"].names 

Map:   0%|          | 0/5871 [00:00<?, ? examples/s]

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(
    huggingface_modelname,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id
    )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
torch.mps.current_allocated_memory(), torch.mps.driver_allocated_memory()

(0, 393216)

In [16]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    learning_rate=learning_rate,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=gradient_accumulation_steps,
    load_best_model_at_end=True,
    )

In [17]:
metric = evaluate.load("accuracy")

In [18]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42),
    eval_dataset=tokenized_datasets["calibration"].shuffle(seed=42),
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

  0%|          | 0/2455 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/164 [00:00<?, ?it/s]

{'eval_loss': 0.7499797344207764, 'eval_accuracy': 0.8337791364157432, 'eval_runtime': 25.3005, 'eval_samples_per_second': 103.437, 'eval_steps_per_second': 6.482, 'epoch': 1.0}
{'loss': 2.4354, 'learning_rate': 3.185336048879837e-05, 'epoch': 1.02}


  0%|          | 0/164 [00:00<?, ?it/s]

...

  0%|          | 0/164 [00:00<?, ?it/s]

{'eval_loss': 0.38200643658638, 'eval_accuracy': 0.9159342758884219, 'eval_runtime': 21.7475, 'eval_samples_per_second': 120.336, 'eval_steps_per_second': 7.541, 'epoch': 4.0}
{'loss': 0.0826, 'learning_rate': 7.413441955193483e-06, 'epoch': 4.07}


  0%|          | 0/164 [00:00<?, ?it/s]

{'eval_loss': 0.3760813772678375, 'eval_accuracy': 0.92281238058846, 'eval_runtime': 21.5999, 'eval_samples_per_second': 121.158, 'eval_steps_per_second': 7.593, 'epoch': 5.0}
{'train_runtime': 1642.0327, 'train_samples_per_second': 23.9, 'train_steps_per_second': 1.495, 'train_loss': 0.6489931988376221, 'epoch': 5.0}


TrainOutput(global_step=2455, training_loss=0.6489931988376221, metrics={'train_runtime': 1642.0327, 'train_samples_per_second': 23.9, 'train_steps_per_second': 1.495, 'train_loss': 0.6489931988376221, 'epoch': 5.0})

In [21]:
pred_train = trainer.predict(tokenized_datasets["train"])
pred_val = trainer.predict(tokenized_datasets["calibration"])
pred_test = trainer.predict(tokenized_datasets["test"])

pred_atis = trainer.predict(tokenized_atis)

  0%|          | 0/491 [00:00<?, ?it/s]

  0%|          | 0/164 [00:00<?, ?it/s]

  0%|          | 0/164 [00:00<?, ?it/s]

  0%|          | 0/367 [00:00<?, ?it/s]

In [22]:
train_sm = scipy.special.softmax(pred_train.predictions, axis=1)
val_sm = scipy.special.softmax(pred_val.predictions, axis=1)
test_sm = scipy.special.softmax(pred_test.predictions, axis=1)

atis_sm = scipy.special.softmax(pred_atis.predictions, axis=1)


np.testing.assert_allclose(train_sm.sum(axis=1), 1.0, rtol=1e-6)
np.testing.assert_allclose(val_sm.sum(axis=1), 1.0, rtol=1e-6)
np.testing.assert_allclose(test_sm.sum(axis=1), 1.0, rtol=1e-6)
np.testing.assert_allclose(atis_sm.sum(axis=1), 1.0, rtol=1e-6)

In [35]:
# train_sm_dict = get_score(train_sm, oos_label)
# val_sm_dict = get_score(val_sm, oos_label)
# test_sm_dict = get_score(test_sm, oos_label)
train_sm_dict = {
    f'label_{i}':  train_sm[:,i] for i in range(train_sm.shape[1])
}
val_sm_dict = {
    f'label_{i}':  val_sm[:,i] for i in range(val_sm.shape[1])
}
test_sm_dict = {
    f'label_{i}':  test_sm[:,i] for i in range(test_sm.shape[1])
}

atis_sm_dict = {
    f'label_{i}':  atis_sm[:,i] for i in range(atis_sm.shape[1])
}

train_frame = pd.DataFrame(
    {"text": dataset["train"]["text"],
     "hash": range(len(dataset["train"]["text"])),
     "label": dataset["train"]["labels"],
     **train_sm_dict
    }
)
val_frame = pd.DataFrame(
    {"text": dataset["calibration"]["text"],
     "hash": range(len(dataset["calibration"]["text"])),
     "label": dataset["calibration"]["labels"],
     **val_sm_dict
    }
)
test_frame = pd.DataFrame(
    {"text": dataset["test"]["text"],
     "hash": range(len(dataset["test"]["text"])),
     "label": dataset["test"]["labels"],
     **test_sm_dict
    }
)
atis_frame = pd.DataFrame(
    {"text": atis_full["text"],
     "hash": range(len(atis_full["text"])),
     "label": ([max(id2label) + 1] * len(atis_full)),
     **atis_sm_dict
    }
)

In [36]:
train_frame.to_csv('data/banking77/bert/train_predictions.csv', index=False)
val_frame.to_csv('data/banking77/bert/calibration_predictions.csv', index=False)
test_frame.to_csv('data/banking77/bert/test_predictions.csv', index=False)
atis_frame.to_csv('data/banking77/bert/atis_predictions.csv', index=False)

pd.DataFrame({'0': id2label.values()}).to_csv('data/banking77/bert/labels.csv')
